In [182]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import fuzzywuzzy # formator data
from fuzzywuzzy import process
from sklearn.impute import SimpleImputer # to change nan to mean value
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vehicle-sales-data/car_prices.csv


# Setup
Importing data

In [183]:
vehicle_location = '../input/vehicle-sales-data/car_prices.csv'
vehicle_data = pd.read_csv(vehicle_location)

# Showing table top 5 rows

In [184]:
vehicle_data.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


# Droping useless columns

In [185]:
vehicle_data_formated = vehicle_data
vehicle_data_formated.drop(labels = ['interior','seller','trim','saledate','mmr','state'],axis = 1,inplace = True)

# Formating database

## Cleaning transmission

In [186]:
#function to find transmission in columns 
def find_transmission(row):
    for record in row:
        if record == 'automatic' or record == 'manual':
            return record
    return row['transmission']
#apply function and changing data
vehicle_data_formated['transmission'] = vehicle_data.apply(find_transmission, axis=1)
#checking what left
wrong_transmision = vehicle_data_formated[(vehicle_data_formated['transmission'] != 'automatic') & (vehicle_data_formated['transmission'] != 'manual') & (
vehicle_data_formated['transmission'].isna() !=True)]
wrong_transmision

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
461612,2015,Volkswagen,Jetta,Navitgation,Sedan,NaN,NaN,46.0,2711,14250.0
505299,2015,Volkswagen,Jetta,Navitgation,sedan,NaN,NaN,36.0,20379,13600.0
529009,2015,Volkswagen,Jetta,Navitgation,sedan,NaN,NaN,2.0,2817,13750.0
551222,2015,Volkswagen,Jetta,Navitgation,sedan,NaN,NaN,2.0,9562,13200.0


In [187]:
# i'll delete it cuz there is a lot of missing data
# creating index_transmision_to_delete
index_transmision_to_delete = wrong_transmision.index
#deleting rows
vehicle_data_formated.drop(index_transmision_to_delete, inplace=True)

In [188]:
# showing nan in transmision column
vehicle_data_formated[(vehicle_data_formated.transmission.isna())]

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
35,2014,Buick,Verano,Sedan,NaN,1g4pr5sk8e4175320,2.0,19531.0,gray,9200.0
44,2014,Chevrolet,Cruze,Sedan,NaN,1g1pc5sb6e7128803,NaN,38261.0,white,2000.0
89,2014,Chevrolet,Silverado 2500HD,Crew Cab,NaN,1gc1kvc84ef152736,36.0,8742.0,white,34000.0
112,2014,Chevrolet,Cruze,Sedan,NaN,1g1pc5sbxe7113253,28.0,40393.0,gray,11700.0
281,2013,Hyundai,Sonata Hybrid,Sedan,NaN,kmhec4a4xda096513,48.0,1111.0,silver,15800.0
...,...,...,...,...,...,...,...,...,...,...
558814,2014,Dodge,Charger,Sedan,NaN,2c3cdxbg3eh324197,43.0,20485.0,white,16100.0
558820,2014,Dodge,Charger,Sedan,NaN,2c3cdxbg9eh324236,42.0,22744.0,white,15900.0
558829,2012,Hyundai,Elantra,Sedan,NaN,5npdh4ae7ch106397,4.0,66720.0,gray,10400.0
558830,2012,Nissan,Sentra,Sedan,NaN,3n1ab6ap3cl622485,26.0,35858.0,white,10400.0


In [189]:
# # taking all unique value 
# models_arr = vehicle_data_formated.model.unique()
# # main model
# def transmission_nan_changer(df:pd.DataFrame):
#     # lop for every string in array
#     for model_name in models_arr:
#         transmission_type = df[(df.model == model_name) & (df.transmission.isna() == False)].transmission.unique()
#         # indexes to change transmission
#         indexes = df[(df.model == model_name) & (df.transmission.isna() != False)].index
        
#         if len(transmission_type) != 0:
#             df.loc[indexes,'transmission'] = transmission_type[0]
#         else:
#             df.loc[indexes,'transmission'] = 'manual'
#     return df
# transmission_nan_changer(vehicle_data_formated)
# vehicle_data_formated.transmission.unique()


In [190]:
# # idexes of data that left 
# idexes_left_trans = vehicle_data_formated[(vehicle_data_formated.transmission.isna())].index
# vehicle_data_formated.drop(idexes_left_trans, inplace = True)

In [191]:
# showing nan in transmision column
vehicle_data_formated[(vehicle_data_formated.transmission.isna())]

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
35,2014,Buick,Verano,Sedan,NaN,1g4pr5sk8e4175320,2.0,19531.0,gray,9200.0
44,2014,Chevrolet,Cruze,Sedan,NaN,1g1pc5sb6e7128803,NaN,38261.0,white,2000.0
89,2014,Chevrolet,Silverado 2500HD,Crew Cab,NaN,1gc1kvc84ef152736,36.0,8742.0,white,34000.0
112,2014,Chevrolet,Cruze,Sedan,NaN,1g1pc5sbxe7113253,28.0,40393.0,gray,11700.0
281,2013,Hyundai,Sonata Hybrid,Sedan,NaN,kmhec4a4xda096513,48.0,1111.0,silver,15800.0
...,...,...,...,...,...,...,...,...,...,...
558814,2014,Dodge,Charger,Sedan,NaN,2c3cdxbg3eh324197,43.0,20485.0,white,16100.0
558820,2014,Dodge,Charger,Sedan,NaN,2c3cdxbg9eh324236,42.0,22744.0,white,15900.0
558829,2012,Hyundai,Elantra,Sedan,NaN,5npdh4ae7ch106397,4.0,66720.0,gray,10400.0
558830,2012,Nissan,Sentra,Sedan,NaN,3n1ab6ap3cl622485,26.0,35858.0,white,10400.0


## Cleaning vin

In [192]:
def find_vin(row):
    for record in row:
        try:
            if len(record) == 17:
                return record
        except:
            pass
    return row['vin']
vehicle_data_formated['vin'] = vehicle_data.apply(find_vin, axis = 1)

In [193]:
vehicle_data_formated.sample(5)

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
365420,2012,Volkswagen,Jetta,Sedan,automatic,3vwdx7aj0cm462568,42.0,32328.0,black,11100.0
173071,2011,Chevrolet,Malibu,Sedan,automatic,1g1ze5e11bf162624,2.0,117881.0,black,7500.0
500076,2004,Audi,A4,sedan,automatic,waulc68e84a193681,25.0,217396.0,gray,1300.0
280019,2007,Ford,Explorer Sport Trac,Crew Cab,automatic,1fmeu31k67ua12390,19.0,111460.0,white,8250.0
32248,2013,Ford,Explorer,SUV,automatic,1fm5k8gt1dgb83075,44.0,39327.0,black,34100.0


In [194]:
# Checking what left
left_vin =  vehicle_data_formated[vehicle_data_formated.vin.str.len() != 17]
left_vin

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
408161,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,46.0,4802,13200.0
417835,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,1.0,9410,13300.0
421289,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,46.0,1167,13200.0
424161,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,1.0,2172,14050.0
427040,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,41.0,14872,13700.0
427043,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,49.0,12655,13850.0
434424,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,46.0,15719,13650.0
444501,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,2.0,6388,13850.0
453794,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,44.0,16633,13600.0
461597,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,44.0,11034,13900.0


In [195]:
# Deleting whats left 
vehicle_data_formated.drop(left_vin.index, inplace = True)

## Cleaning car makers

In [196]:
vehicle_data_formated = vehicle_data_formated[vehicle_data_formated['make'].isnull() == False]
car_makers = vehicle_data_formated.make.unique()
np.unique(np.array([car.lower() for car in car_makers]))

array(['acura', 'airstream', 'aston martin', 'audi', 'bentley', 'bmw',
       'buick', 'cadillac', 'chev truck', 'chevrolet', 'chrysler',
       'daewoo', 'dodge', 'dodge tk', 'dot', 'ferrari', 'fiat', 'fisker',
       'ford', 'ford tk', 'ford truck', 'geo', 'gmc', 'gmc truck',
       'honda', 'hummer', 'hyundai', 'hyundai tk', 'infiniti', 'isuzu',
       'jaguar', 'jeep', 'kia', 'lamborghini', 'land rover', 'landrover',
       'lexus', 'lincoln', 'lotus', 'maserati', 'mazda', 'mazda tk',
       'mercedes', 'mercedes-b', 'mercedes-benz', 'mercury', 'mini',
       'mitsubishi', 'nissan', 'oldsmobile', 'plymouth', 'pontiac',
       'porsche', 'ram', 'rolls-royce', 'saab', 'saturn', 'scion',
       'smart', 'subaru', 'suzuki', 'tesla', 'toyota', 'volkswagen',
       'volvo', 'vw'], dtype='<U13')

### Cleaning car makers 

In [205]:
def formating_car_makers(brand):
    if isinstance(brand, str):
        mapping = {
            'chev truck': 'chevrolet',
            'dodge tk': 'dodge',
            'gmc truck': 'gmc',
            'hyundai tk': 'hyundai',
            'land rover': 'landrover',
            'mazda tk': 'mazda',
            'mercedes-b': 'mercedes',
            'mercedes-benz': 'mercedes',
            'vw': 'volkswagen'
        }
        brand = brand.lower()
        if brand in mapping:
            return mapping[brand]
        else:
            return brand
    else:
        ''
    
vehicle_data_formated.loc[:,'make'] = vehicle_data.make.apply(lambda brand: formating_car_makers(brand))
vehicle_data_formated = vehicle_data_formated[vehicle_data_formated['make'] != 'dot']

## Cleaning car models

In [198]:
vehicle_data_formated.sample(5)

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
85413,2013.0,dodge,Avenger,Sedan,automatic,1c3cdzag7dn717560,25.0,31275.0,blue,8700.0
441163,2012.0,nissan,Altima,Sedan,automatic,1n4al2ap0cn460078,28.0,53675.0,black,11800.0
364767,2012.0,infiniti,G Coupe,G Coupe,automatic,jn1cv6ek3cm420704,45.0,27127.0,black,23200.0
468160,2013.0,ford,Focus,sedan,automatic,1fadp3f21dl237467,32.0,55837.0,red,9600.0
420672,2013.0,chevrolet,Malibu,Sedan,NaN,1g11c5sa8df285900,31.0,51879.0,silver,12000.0


In [199]:
vehicle_data_formated.loc[:,'model'] = vehicle_data_formated.model.str.lower()
vehicle_data_formated.loc[:,'model'] = vehicle_data_formated.model.str.strip()


## Cleaning body

In [200]:
vehicle_data_formated.loc[:,'body'] = vehicle_data_formated.body.str.lower()
vehicle_data_formated.loc[:,'body'] = vehicle_data_formated.body.str.strip()

In [201]:
db_to_count = vehicle_data_formated.copy()
db_to_count['make and model'] = db_to_count['make'] + ' ' + db_to_count['model']
db_to_count[db_to_count.body.isna()]

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice,make and model
468,2013.0,lincoln,mkt,NaN,automatic,2lmhj5nk1dbl52290,41.0,74874.0,black,17750.0,lincoln mkt
743,2012.0,bmw,750i,NaN,automatic,wbakc6c5xcc395623,4.0,50790.0,gray,33500.0,bmw 750i
770,2012.0,bmw,750li,NaN,automatic,wbakb8c50cc963182,37.0,31762.0,black,45000.0,bmw 750li
793,2012.0,bmw,750i,NaN,automatic,wbaka8c54ccy37992,49.0,53016.0,white,34250.0,bmw 750i
794,2012.0,bmw,750li,NaN,automatic,wbakb8c56cc963431,34.0,24739.0,white,46750.0,bmw 750li
...,...,...,...,...,...,...,...,...,...,...,...
541919,2004.0,honda,accord,NaN,automatic,1hgcm66884a076293,35.0,199124.0,burgundy,3200.0,honda accord
548316,2010.0,chevrolet,malibu,NaN,automatic,1g1zb5eb7af274800,33.0,73483.0,gray,7400.0,chevrolet malibu
548323,2010.0,chevrolet,malibu,NaN,automatic,1g1zb5eb9af208345,33.0,59853.0,black,7200.0,chevrolet malibu
550571,2010.0,chevrolet,malibu,NaN,automatic,1g1zb5eb4af307705,28.0,76282.0,white,7400.0,chevrolet malibu


In [202]:
db_to_count[db_to_count.body.isna()]['make and model'].value_counts()

make and model
chrysler pacifica        148
mitsubishi galant        100
lexus gx                  98
chevrolet hhr             92
honda pilot               82
                        ... 
chevrolet cavalier         1
ford 350                   1
landrover rrs              1
ford truck expedition      1
chevrolet uplandr          1
Name: count, Length: 220, dtype: int64

In [203]:
def changer_make(make, model, body):
    indexes = vehicle_data_formated[(vehicle_data_formated.make == make) & (vehicle_data_formated.model == model)].index
    vehicle_data_formated.loc[indexes,'body'] = body

changer_make('chevrolet', 'malibu', 'sedan')
changer_make('bmw', '750i', 'sedan')
changer_make('chrysler', 'pacifica', 'minivan')
changer_make('lexus', 'gx', 'suv')
changer_make('chevrolet', 'hhr', 'suv')
changer_make('honda', 'pilot', 'suv')
changer_make('landrover', 'range', 'suv')
changer_make('cadillac', 'srx', 'suv')
changer_make('ford', 'explorer', 'suv')
changer_make('ford', 'f150', 'pickup')
changer_make('chevrolet', 'corvette', 'coupe')

In [207]:
indexes_to_drop_body = vehicle_data_formated[vehicle_data_formated.body.isna()].index
vehicle_data_formated.drop(indexes_to_drop_body,inplace = True)
vehicle_data_formated

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
0,2015.0,kia,sorento,suv,automatic,5xyktca69fg566472,5.0,16639.0,white,21500.0
1,2015.0,kia,sorento,suv,automatic,5xyktca69fg561319,5.0,9393.0,white,21500.0
2,2014.0,bmw,3 series,sedan,automatic,wba3c1c51ek116351,45.0,1331.0,gray,30000.0
3,2015.0,volvo,s60,sedan,automatic,yv1612tb4f1310987,41.0,14282.0,white,27750.0
4,2014.0,bmw,6 series gran coupe,sedan,automatic,wba6b2c57ed129731,43.0,2641.0,gray,67000.0
...,...,...,...,...,...,...,...,...,...,...
558832,2015.0,kia,k900,sedan,NaN,knalw4d4xf6019304,45.0,18255.0,silver,33000.0
558833,2012.0,ram,2500,crew cab,automatic,3c6td5et6cg112407,5.0,54393.0,white,30800.0
558834,2012.0,bmw,x5,suv,automatic,5uxzw0c58cl668465,48.0,50561.0,black,34000.0
558835,2015.0,nissan,altima,sedan,automatic,1n4al3ap0fc216050,38.0,16658.0,white,11100.0
